# Note: this file using predefined models for simplicity

# CIFAR-10: Training and Testing on a Clean Dataset & Adversarial Analysis

## Imports and CIFAR-10 loading

In [1]:
import models.resnet as resnet
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as DataUtils
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm.notebook import tnrange, tqdm

In [2]:
# Define the `device` PyTorch will be running on, please hope it is CUDA
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Notebook will use PyTorch Device: " + device.upper())

Notebook will use PyTorch Device: CUDA


In [3]:
DATA_ROOT = "./datasets/"


def get_CIFAR10_data_loaders(
    batchSize=64, trainSetSize=50000, validationSetSize=10000, testSetSize=10000
):

    # Create a separate transform for each dataset
    # (in case we decide to transform differently)
    trainSetTransform = transforms.Compose([transforms.ToTensor()])
    validationSetTransform = transforms.Compose([transforms.ToTensor()])
    testSetTransform = transforms.Compose([transforms.ToTensor()])

    # Download the dataset (note we technically use the same set for validation
    # and training)
    trainSet = datasets.CIFAR10(
        root=DATA_ROOT, download=True, train=True, transform=trainSetTransform
    )
    validationSet = datasets.CIFAR10(
        root=DATA_ROOT, download=True, train=True, transform=validationSetTransform
    )
    testSet = datasets.CIFAR10(
        root=DATA_ROOT, download=True, train=False, transform=testSetTransform
    )

    # Get the training indices to split into training and validation sets
    indices = np.arange(0, trainSetSize + validationSetSize)
    np.random.shuffle(indices)

    # Construct random samplers (for better training)
    trainSetSampler = SubsetRandomSampler(indices[:trainSetSize])
    validationSetSampler = SubsetRandomSampler(indices[trainSetSize:])
    testSetSampler = SubsetRandomSampler(np.arange(0, testSetSize))

    # Finally, construct the loaders that will be used to get images
    trainSetLoader = DataUtils.DataLoader(
        trainSet, batch_size=batchSize, sampler=trainSetSampler
    )
    validationSetLoader = DataUtils.DataLoader(
        validationSet, batch_size=batchSize, sampler=validationSetSampler
    )
    testSetLoader = DataUtils.DataLoader(
        testSet, batch_size=batchSize, sampler=testSetSampler
    )

    # Return the loaders
    return trainSetLoader, validationSetLoader, testSetLoader

## Training, Validation and Testing

In [4]:
# Get the data loaders (assume we do no validation)
trainSetLoader, _, testSetLoader = get_CIFAR10_data_loaders(
    trainSetSize=50000, validationSetSize=0
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [5]:
# This method creates a new model and also trains it
def get_trained_model():
    # Various training parameters
    epochs = 60
    learning_rate = 0.01

    # Network parameters
    loss_function = nn.CrossEntropyLoss()
    model = resnet.ResNet50()
    model = model.to(device)
    model.train()

    # Consider using ADAM here as another gradient descent algorithm
    optimizer = torch.optim.SGD(
        model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4
    )

    print("Training the model...")

    # Use a pretty progress bar to show updates
    for epoch in tnrange(epochs, desc="Training Progress"):
        for _, (images, labels) in enumerate(tqdm(trainSetLoader, desc="Batches")):
            # Cast to proper tensors
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            logits = model(images)

            loss = loss_function(logits, labels)
            loss.backward()

            optimizer.step()

    print("... done!")

    # Return the trained model
    return model

In [6]:
# This method tests a given mode and prints the accuracy of that model
def test_trained_model(model):
    correct = 0

    print("Testing the model...")

    # Use a pretty progress bar to show updates
    for j, (images, labels) in enumerate(tqdm(testSetLoader, desc="Testing Progress")):
        # Cast to proper tensor
        images, labels = images.to(device), labels.to(device)

        # Predict
        logits = model(images)

        # The highest class represents the chosen class
        _, preds = torch.max(logits, 1)
        correct += (preds == labels).sum().item()

    print("... done! Accuracy: {}%".format(float(correct) * 100 / 10000))

## Putting it all together (the 'main')

In [7]:
model = get_trained_model()

test_trained_model(model)

Training the model...


Training Progress:   0%|          | 0/60 [00:00<?, ?it/s]

Batches:   0%|          | 0/782 [00:00<?, ?it/s]

Batches:   0%|          | 0/782 [00:00<?, ?it/s]

Batches:   0%|          | 0/782 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/782 [00:00<?, ?it/s]

Batches:   0%|          | 0/782 [00:00<?, ?it/s]

... done!
Testing the model...


Testing Progress:   0%|          | 0/157 [00:00<?, ?it/s]

... done! Accuracy: 83.64%


In [8]:
# Save the model
torch.save(model, "cifar10_model")

In [10]:
# Load and test the loaded model (shortcutting many steps)
model = torch.load("cifar10_model")

test_trained_model(model)

Testing the model...


Testing Progress:   0%|          | 0/157 [00:00<?, ?it/s]

... done! Accuracy: 83.53%
